In [19]:
import random
import time

class TransactionType:
  OP_BUY = 1
  OP_SELL = -1

class TransactionOrder:
  def __init__(self, market, party, transactionType, assetName, share, price):
    self.market = market
    self.party = party
    self.transactionType = transactionType
    self.assetName = assetName
    self.share = share
    self.price = price
    self.timestamp = int(time.mktime(time.gmtime()))
    self.status = "init" # including reject, timeout, deal, init, cancel

  def __repr__(self):
    return str({
      "market": self.market,
      "party": self.party.userId,
      "transactionType": self.transactionType,
      "assetName": self.assetName,
      "share": self.share,
      "price": self.price,
      "timestamp": str(self.timestamp),
      "status": self.status
    })

class Market:
  def __init__(self, marketName):
    self.marketName = marketName
    self.users = []
    self.assets = {}
    self.transactionOrders = {}
  
  def transact(self, buyer, seller, assetName, share, price):
    assetsAtPrice, dealAmount = seller.sell(self, assetName, share, price)
    if len(assetsAtPrice.keys()) >0:
      buyer.buy(self, assetName, assetsAtPrice["share"], price)
  
  def register(self, user):
    self.users.append(user)
  
  def issue(self, asset):
    self.assets[asset.assetName] = asset

  def __repr__(self):
    return str({
    })
  
  def request_transaction(self, transactionOrder):
    transactionOrder.market = self
    if assetName in self.transactionOrders:
      self.transactionOrders[assetName].append(transactionOrder)
    else:
      self.transactionOrders[assetName] = [transactionOrder]
  
  def match(self):
    # loop through all assets
    for assetName in self.transactionOrders.keys():
      # sort all the transactionOrder
      self.transactionOrders[assetName] = sorted(self.transactionOrders[assetName], key=lambda o: o.transactionType * o.price)
      buyOrders, sellOrders = [], []
      for i in range(len(self.transactionOrders[assetName])):
        if self.transactionOrders[assetName][i].transactionType == TransactionType.OP_BUY:
          buyOrders.append(self.transactionOrders[assetName][i])
        else:
          sellOrders.append(self.transactionOrders[assetName][i])
      if len(buyOrders) ==0 or len(sellOrders) ==0:
        # no liquidity
        continue
      else:
        #while True:
        #  for i in range()
        # not finished yet
        pass

class Asset:
  def __init__(self, assetName, share):
    self.assetName = assetName
    self.share = share
    self.historyPrice = []

  def __repr__(self):
    return str({
      "assetName":self.assetName,
      "share":self.share
    })

class User:
  userId = 0
  def __init__(self, username=None):
    if username == None:
      self.username = "user_%d" % User.userId
    else:
      self.username = username
    self.userId = User.userId
    User.userId += 1
    self.assets = {}
    self.cash = 0.
    self.active = True
  
  def distribute(self, market, assetName, share):
    if assetName in self.assets:
      self.assets[assetName] += share
    else:
      self.assets[assetName] = share

  def buy(self, market, assetName, share, price):
    self.cash -= float(share) * price
    if assetName in self.assets:
      self.assets[assetName] += share
    else:
      self.assets[assetName] = share
  
  def sell(self, market, assetName, share, price):
    if assetName in self.assets and self.assets[assetName] > 0:
      # to support partial deal, for better liquidity
      shareToDeal = min(share, self.assets[assetName])
      self.assets[assetName] -= shareToDeal
      dealAmount = float(shareToDeal) * price # - tax
      self.cash += dealAmount
      assetAtPrice = {"assetName": assetName, "share": shareToDeal}
      return assetAtPrice, dealAmount
    else:
      return {}, 0
  
  def __repr__(self):
    return str({
      "username": self.username,
      "userId": self.userId,
      "cash": self.cash,
      "assets": self.assets
    })

In [17]:
# setup market and users
stockMarket = Market("stockMarket")
for i in range(10):
  user = User()
  user.cash = 100000. #float(random.randint(100, 10000)) * 100.
  stockMarket.register(user)

# issue assets
ASSET_NAME="stock000"
stockMarket.issue(Asset(ASSET_NAME, 5000))
distributed = 0
totalShare = stockMarket.assets[ASSET_NAME].share
assetName = stockMarket.assets[ASSET_NAME].assetName
distributedShares = [1000, 2000, 2000]
for i in range(3):
  share = distributedShares[i]
  user.distribute(stockMarket, assetName, share)
  distributed += share
  user = stockMarket.users[i]
  user.cash = 0.

# start to trade
stockMarket.request_transaction(TransactionOrder(stockMarket, stockMarket.users[5], TransactionType.OP_BUY, ASSET_NAME, 500, 15.))
stockMarket.request_transaction(TransactionOrder(stockMarket, stockMarket.users[5], TransactionType.OP_BUY, ASSET_NAME, 500, 16.5))
stockMarket.request_transaction(TransactionOrder(stockMarket, stockMarket.users[6], TransactionType.OP_BUY, ASSET_NAME, 1000, 17.))

stockMarket.request_transaction(TransactionOrder(stockMarket, stockMarket.users[0], TransactionType.OP_SELL, ASSET_NAME, 500, 15.))
stockMarket.request_transaction(TransactionOrder(stockMarket, stockMarket.users[0], TransactionType.OP_SELL, ASSET_NAME, 500, 16.5))
stockMarket.request_transaction(TransactionOrder(stockMarket, stockMarket.users[0], TransactionType.OP_SELL, ASSET_NAME, 1000, 17.))

stockMarket.match()

# display the current status
for i in range(10):
  print(stockMarket.users[i])

{'market': {}, 'party': 0, 'transactionType': -1, 'assetName': 'stock000', 'share': 1000, 'price': 17.0, 'timestamp': '1618799439', 'status': 'init'}
{'market': {}, 'party': 0, 'transactionType': -1, 'assetName': 'stock000', 'share': 500, 'price': 16.5, 'timestamp': '1618799439', 'status': 'init'}
{'market': {}, 'party': 0, 'transactionType': -1, 'assetName': 'stock000', 'share': 500, 'price': 15.0, 'timestamp': '1618799439', 'status': 'init'}
{'market': {}, 'party': 5, 'transactionType': 1, 'assetName': 'stock000', 'share': 500, 'price': 15.0, 'timestamp': '1618799439', 'status': 'init'}
{'market': {}, 'party': 5, 'transactionType': 1, 'assetName': 'stock000', 'share': 500, 'price': 16.5, 'timestamp': '1618799439', 'status': 'init'}
{'market': {}, 'party': 6, 'transactionType': 1, 'assetName': 'stock000', 'share': 1000, 'price': 17.0, 'timestamp': '1618799439', 'status': 'init'}
{'username': 'user_0', 'userId': 0, 'cash': 0.0, 'assets': {'stock000': 2000}}
{'username': 'user_1', 'user